In [1]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")

In [2]:
from AlphaNet.Data import DataLoader

In [3]:
dataloader = DataLoader()

In [4]:
dataloader.load_data_from_file(data_path="/home/wuwenjun/Data_lib/ti0/wuwenjun/test/",end_date="2022-01-01")

test_mcorr_open_low_10    test_1     test  \
timestamp           ticker                                              
2020-01-02 09:30:00 000001                0.115025  0.115025  3604.78   
                    000002                0.796874  0.796874  9428.61   
                    000004                0.815852  0.815852   184.01   
                    000005                0.815519  0.815519    57.28   
                    000006                0.846951  0.846951   379.05   
...                                            ...       ...      ...   
2021-06-01 09:30:00 688699                     NaN       NaN      NaN   
                    688777                     NaN       NaN      NaN   
                    688788                     NaN       NaN      NaN   
                    688819                     NaN       NaN      NaN   
                    688981                     NaN       NaN      NaN   

                            ewmCorr(high,low)  ewmCorr(high,low,7)    target  
timestamp           ticker                                                    
2020-01-02 09:30:00 000001           0.414245             0.414245 -5.551395  
                    000002           0.968666             0.968666 -6.203258  
                    000004          -0.086929            -0.086929 -5.390784  
                    000005           0.656316             0.656316 -5.142255  
                    000006           0.967827             0.967827 -5.327572  
...                                       ...                  ...       ...  
2021-06-01 09:30:00 688699                NaN                  NaN -6.236975  
                    688777                NaN                  NaN -6.432447  
                    688788                NaN                  NaN -6.486812  
                    688819                NaN                  NaN -6.124710  
                    688981                NaN                  NaN -6.088935  

[1366034 rows x 6 columns]

In [10]:
shift = 7

In [18]:
sequence_list = [0]
for i in range(shift-1,30,shift):
    sequence_list.append(i)
sequence_list.sort(reverse=True)
sequence_list

[27, 20, 13, 6, 0]

In [17]:
x = dataloader.feature_data.drop("target",axis=1)
x

test_mcorr_open_low_10    test_1     test  \
timestamp           ticker                                              
2020-01-02 09:30:00 000001                0.115025  0.115025  3604.78   
                    000002                0.796874  0.796874  9428.61   
                    000004                0.815852  0.815852   184.01   
                    000005                0.815519  0.815519    57.28   
                    000006                0.846951  0.846951   379.05   
...                                            ...       ...      ...   
2021-06-01 09:30:00 688699                     NaN       NaN      NaN   
                    688777                     NaN       NaN      NaN   
                    688788                     NaN       NaN      NaN   
                    688819                     NaN       NaN      NaN   
                    688981                     NaN       NaN      NaN   

                            ewmCorr(high,low)  ewmCorr(high,low,7)  
timestamp           ticker                                          
2020-01-02 09:30:00 000001           0.414245             0.414245  
                    000002           0.968666             0.968666  
                    000004          -0.086929            -0.086929  
                    000005           0.656316             0.656316  
                    000006           0.967827             0.967827  
...                                       ...                  ...  
2021-06-01 09:30:00 688699                NaN                  NaN  
                    688777                NaN                  NaN  
                    688788                NaN                  NaN  
                    688819                NaN                  NaN  
                    688981                NaN                  NaN  

[1366034 rows x 5 columns]

In [21]:
final_df = []
for index,group in tqdm(x.groupby("ticker")):
    group_df = []
    for i in sequence_list:
        a = group.shift(i)
        a.columns = [str(j)+"_Shift_%i" % i for j in group.columns]
        group_df.append(a)
    group_df = pd.concat(group_df,axis=1)
    final_df.append(group_df)
final_df = pd.concat(final_df,axis=0)
final_df

100%|██████████| 4353/4353 [00:10<00:00, 398.29it/s]


,,test_mcorr_open_low_10_Shift_27,test_1_Shift_27,test_Shift_27,"ewmCorr(high,low)_Shift_27","ewmCorr(high,low,7)_Shift_27",test_mcorr_open_low_10_Shift_20,test_1_Shift_20,test_Shift_20,"ewmCorr(high,low)_Shift_20","ewmCorr(high,low,7)_Shift_20",...,test_mcorr_open_low_10_Shift_6,test_1_Shift_6,test_Shift_6,"ewmCorr(high,low)_Shift_6","ewmCorr(high,low,7)_Shift_6",test_mcorr_open_low_10_Shift_0,test_1_Shift_0,test_Shift_0,"ewmCorr(high,low)_Shift_0","ewmCorr(high,low,7)_Shift_0"
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2020-01-02 09:30:00,000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.115025,0.115025,3604.78,0.414245,0.414245
2020-01-03 09:30:00,000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.340535,0.340535,3659.36,0.922935,0.922935
2020-01-06 09:30:00,000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.717011,0.717011,3724.86,0.989700,0.989700
2020-01-07 09:30:00,000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.820196,0.820196,3720.49,0.987041,0.987041
2020-01-08 09:30:00,000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.931442,0.931442,3742.33,0.893703,0.893703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-26 09:30:00,688981,0.972941,0.972941,111.92,0.870913,0.870913,0.886539,0.886539,110.29,0.934368,0.934368,...,0.910865,0.910865,110.70,0.967554,0.967554,0.954971,0.954971,110.85,0.900790,0.900790
2021-05-27 09:30:00,688981,0.968343,0.968343,112.47,0.918865,0.918865,0.883308,0.883308,110.72,0.736020,0.736020,...,0.910914,0.910914,110.18,0.933708,0.933708,0.941155,0.941155,110.71,0.924924,0.924924
2021-05-28 09:30:00,688981,0.958937,0.958937,113.17,0.897014,0.897014,0.883115,0.883115,109.82,-0.545963,-0.545963,...,0.906171,0.906171,109.59,0.910664,0.910664,0.935656,0.935656,111.88,0.083880,0.083880
